In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from unidecode import unidecode
import numpy as np


# Pré-processamento dos dados
- Esta célula lê o banco de dados de um arquivo .xlsx, este arquivo deve conter pelo menos duas colunas:
    - Uma chamada Descrição
    - Outra chamada relevância (0 para não relevante, 1 para relevante)
- Stopwords
    - Stopwords são palavras que não tem significado substancial para o aprendizado de máquina, palavras como:
        - O, de, el, de la, etc...
    - Após isso, é necessário baixar os stopwords, para que sejam removidos da Descrição para aumentar a eficiência da interpretação do algoritmo
        - Stopwords em português e espanhol
    - Este passo deve ser feito apenas na primeira execução
- É definido uma função para extrair as stopwords da Descrição, remoção de todos os acentos e caracteres especiais, e definir todas as letras como minúsculas
- Cria-se uma nova coluna no dataframe com as Descrições Processadas

In [2]:
# Carrega banco de dados para treino e teste
data = pd.read_excel("banco_dados.xlsx")

# Baixe as stopwords e o tokenizador da NLTK (necessário apenas na primeira execução)
nltk.download('stopwords')
nltk.download('punkt')
# Inicialize o stemmer e as stopwords
stemmer = PorterStemmer()
# Stepwords em português
stop_words = set(stopwords.words('portuguese'))
# Adicionar stopwords para espanhol
stop_words.update(stopwords.words('spanish')) 	

# Pré-processamento dos dados
def preprocess_text(text):
    # Remoção de pontuações e caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)
    # Conversão para minúsculas
    text = text.lower()
    # Remoção de acentos
    text = unidecode(text)
    # Tokenização
    tokens = word_tokenize(text)
    # Remoção de stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Junte as palavras novamente em uma string
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

# Aplicar a função de pré-processamento à coluna de descriçãoa
data['Descrição Processada'] = data['Descrição'].apply(preprocess_text)
display((data[['Descrição', 'Descrição Processada','Relevância']]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brun7530\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\brun7530\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Descrição,Descrição Processada,Relevância
0,Solicitado el inicio de las maniobras de aisla...,solicitado inicio maniobras aislacion conforme...,0
1,"Entrada de la ATE 202307136 para ""Verificar e ...",entrada ate 202307136 verificar solucionar osc...,0
2,"Entrada de la ATE 202307508 para ""efectuar man...",entrada ate 202307508 efectuar mantenimiento p...,0
3,"Conforme IOC_01.002-17, realizada prueba de pr...",conforme ioc_0100217 realizada prueba preparac...,0
4,"Dentro del PD 20230128, accionado partida de l...",dentro pd 20230128 accionado partida unidad gi...,0
...,...,...,...
5708,"Baixa da ATO 202306261 ""Verificar y solucionar...",baixa ato 202306261 verificar solucionar motiv...,0
5709,"Atendendo solicitação do sistema, acionado com...",atendendo solicitacao sistema acionado comando...,0
5710,"Informado por el Despacho de Carga - Bruno, el...",informado despacho carga bruno termino siguien...,1
5711,"Baixa da ATO 202306201 ""Verificar e soluciona...",baixa ato 202306201 verificar solucionar acion...,0


### Vetorização utilizando TF-IDF (Term Frequency-Inverse Document Frequency)
1. **Entendimento do Conceito**:
   - TF-IDF (Term Frequency-Inverse Document Frequency) é uma técnica para atribuir pesos a cada palavra em um documento com base em sua frequência e sua importância relativa em todo o corpus (conjunto de todos os documentos).
   - O objetivo é representar cada documento como um vetor numérico, onde cada componente do vetor corresponde ao peso de uma palavra específica.

2. **Cálculo da Frequência dos Termos (TF)**:
   - Para cada palavra em um documento, calcula-se a frequência do termo, ou seja, quantas vezes a palavra aparece no documento.
   - A fórmula para calcular a frequência do termo é:
     $$
     TF_{t,d} = \frac{\text{Número de vezes que o termo } t \text{ aparece em } d}{\text{Número total de termos em } d}
     $$
   - Isso resulta em um valor entre 0 e 1, indicando a frequência relativa da palavra no documento.

3. **Cálculo da Frequência do Documento Inverso (IDF)**:
   - Calcula-se o inverso da frequência de documentos que contêm o termo.
   - A fórmula para calcular o IDF é:
     $$
     IDF_t = \log\left(\frac{\text{Número total de documentos}}{\text{Número de documentos que contêm o termo } t}\right)
     $$
   - O logaritmo é usado para penalizar termos que aparecem em muitos documentos.

4. **Cálculo do Produto TF-IDF**:
   - O produto TF-IDF de uma palavra é o produto da frequência do termo no documento (TF) e o inverso da frequência do documento (IDF).
   - A fórmula é:
     $$
     TF-IDF_{t,d} = TF_{t,d} \times IDF_t
     $$
   - Isso resulta em um valor que representa a importância da palavra no contexto do documento e do corpus.

5. **Implementação usando TfidfVectorizer**:
   - A classe TfidfVectorizer do scikit-learn implementa a vetorização de texto usando a técnica TF-IDF.
   - Passos:
     - Criação do vetorizador: Instancie um objeto TfidfVectorizer.
     - Ajuste do vetorizador: Use o método fit_transform para ajustar o vetorizador aos dados de treinamento. Isso aprenderá o vocabulário do corpus e calculará os pesos TF-IDF para cada palavra.
     - Vetorização dos dados: Use o método transform para transformar os dados de treinamento e teste em vetores TF-IDF.

6. **Utilização dos Vetores TF-IDF**:
   - Os vetores TF-IDF resultantes são usados como entrada para modelos de aprendizado de máquina, que podem ser treinados e usados para fazer previsões.

In [3]:
# Divisão dos dados em conjuntos de treinamento e teste
X = data["Descrição Processada"]
y = data["Relevância"]
# 80% treino, 20% teste, 42 para proporção ser randômica e constante, são DataFrames
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vetorização do texto usando TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Você pode converter a matriz esparsa em um DataFrame
df_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

**Instanciação do Modelo de Regressão Logística**:
- Inicialmente é criado uma instância do modelo de Regressão Logística usando a classe *LogisticRegression()* do scikit-learn. Este modelo será usado para fazer previsões sobre a relevância das descrições.

**Treinamento do Modelo**:
- Após isso é ralizado o treinamento do modelo de regressão logística usando o método *fit()*. Este método recebe dois argumentos:
  - *X_train_tfidf*: Este é o conjunto de dados de treinamento pré-processado e vetorizado usando o TF-IDF. Ele contém os vetores TF-IDF das descrições de treinamento.
  - *y_train*: Este é o vetor de rótulos correspondentes aos dados de treinamento. Cada rótulo indica se a descrição correspondente é relevante (1) ou não relevante (0).
- Durante o treinamento, o modelo ajusta os parâmetros (coeficientes) de forma a minimizar a função de perda, que é geralmente a função de entropia cruzada. Isso é feito usando técnicas de otimização, como o gradiente descendente, para ajustar os parâmetros na direção que reduz a função de perda.


**Previsões no conjunto de teste e avaliação do modelo**:
- Aqui, estamos utilizando o modelo treinado para fazer previsões sobre o conjunto de teste. O método *predict()* é aplicado ao modelo, passando como entrada os vetores TF-IDF do conjunto de teste (*X_test_tfidf*), lembrando que este X é o resultado da vetorização das Descrições teste do Banco de dados. Isso retorna as previsões do modelo para cada descrição de teste.

- Após isso, é avaliado o desempenho do modelo utilizando métricas de avaliação. é utilizado a função *accuracy_score()* do scikit-learn para calcular a acurácia do modelo. Ela compara as previsões feitas com os rótulos verdadeiros do conjunto de teste *y_test*. Essa métrica nos dá uma medida de quão preciso é o modelo em suas previsões.


In [4]:

# Treinamento do modelo de Regressão Logística, modelo categórico (relevante ou não)
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Fazendo previsões no conjunto de teste
predictions = model.predict(X_test_tfidf)

# Avaliando o modelo para banco de dados original
accuracy = accuracy_score(y_test, predictions)
print("Acurácia do modelo:", accuracy)
print("\nRelatório de classificação:")
print(classification_report(y_test, predictions))

Acurácia do modelo: 0.8285214348206474

Relatório de classificação:
              precision    recall  f1-score   support

           0       0.87      0.87      0.87       742
           1       0.76      0.76      0.76       401

    accuracy                           0.83      1143
   macro avg       0.81      0.81      0.81      1143
weighted avg       0.83      0.83      0.83      1143



**Teste para outras ocorrências**

- Existe um banco_testes, com as ocorrências relevantes de outras datas para avaliar a aderência do modelo a outros dados

In [5]:
# Testa o desempenho do modelo com um banco de dados teste
banco_teste = pd.read_excel('banco_testes.xlsx', usecols=['Descrição', 'Relevância'])
banco_teste['Descricao_Processada'] = banco_teste['Descrição'].apply(preprocess_text)
aux = 0

for index, description in enumerate(banco_teste['Descricao_Processada']):
	new_description_tfidf = tfidf_vectorizer.transform([description])
	prediction_new = model.predict(new_description_tfidf)
	if banco_teste['Relevância'].iloc[index] == prediction_new:
		aux += 1

# Apresenta a aderência do modelo para banco_testes
aderencia = aux*100/len(banco_teste)
print('Eventos acertados', aux)
print('Total de eventos', len(banco_teste))
print('Aderência do modelo', aderencia)

Eventos acertados 191
Total de eventos 233
Aderência do modelo 81.97424892703863
